In [ ]:
!pip install evaluate datasets

In [ ]:
import re
import torch
import evaluate
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from datasets import load_dataset

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import sys
sys.path.append('../')
from scripts.utils import preprocess_article, calculate_length, tokenize_function, compute_metrics_eval, evaluate_model

In [ ]:
# Load dataset
data = load_dataset("csv", data_files="data/Amharic_corpus_merged_2023-04-16.csv")
data

DatasetDict({
    train: Dataset({
        features: ['article', 'category'],
        num_rows: 61915
    })
})

In [ ]:
# Preprocess and calculate length
data['train'] = data['train'].map(calculate_length, batched=False)
data['train'] = data['train'].map(preprocess_article, batched=False)

In [ ]:
# Split dataset
raw_datasets = data['train'].train_test_split(train_size=0.8, seed=42)
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['article', 'category', 'word_count'],
        num_rows: 49532
    })
    test: Dataset({
        features: ['article', 'category', 'word_count'],
        num_rows: 12383
    })
})

In [ ]:
# Tokenize dataset
categories = list(set(data['train']['category']))
category_to_id = {cat: idx for idx, cat in enumerate(categories)}

model_name = "rasyosef/bert-small-amharic"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenized_datasets = raw_datasets.map(lambda x: tokenize_function(x, tokenizer, category_to_id))

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors='pt')

# Set format for datasets
tokenized_datasets.set_format("torch")

Map:   0%|          | 0/49532 [00:00<?, ? examples/s]

Map:   0%|          | 0/12383 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(categories),
    id2label = {i: lbl for i, lbl in enumerate(categories)},
    label2id = {lbl: i for i, lbl in enumerate(categories)},
    device_map="cuda"
)


embedding_layer = model.base_model.embeddings
print(f"Embedding layer: {embedding_layer}")
print(f"Embedding details: {embedding_layer.word_embeddings.weight.shape}")

print(f"Model configuration: {model.config}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at rasyosef/bert-small-amharic and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding layer: BertEmbeddings(
  (word_embeddings): Embedding(24576, 512, padding_idx=0)
  (position_embeddings): Embedding(512, 512)
  (token_type_embeddings): Embedding(2, 512)
  (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)
Embedding details: torch.Size([24576, 512])
Model configuration: BertConfig {
  "_name_or_path": "rasyosef/bert-small-amharic",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "id2label": {
    "0": "Business",
    "1": "Local News",
    "2": "Entertainment",
    "3": "Sports",
    "4": "Others",
    "5": "Politics",
    "6": "International News"
  },
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "label2id": {
    "Business": 0,
    "Entertainment": 2,
    "International News": 6,
    "Local News": 1,
    "Others": 4,
    "Politics": 5,
 

In [ ]:
training_args = TrainingArguments(
    output_dir=model_name + "-finetuned",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    weight_decay=0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=True,
    seed=42,
)

In [ ]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.593000,0.405836,0.847775,0.848342,0.847775,0.847281
2,0.376400,0.359899,0.864007,0.865583,0.864007,0.863513
3,0.319100,0.348271,0.868449,0.869012,0.868449,0.868034
4,0.281300,0.336950,0.872971,0.872059,0.872971,0.872278
5,0.256500,0.336323,0.876686,0.876218,0.876686,0.876301


TrainOutput(global_step=3870, training_loss=0.36527942588471013, metrics={'train_runtime': 1313.4307, 'train_samples_per_second': 188.56, 'train_steps_per_second': 2.946, 'total_flos': 9796823106416640.0, 'train_loss': 0.36527942588471013, 'epoch': 5.0})

In [ ]:
# Evaluate the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
metrics = evaluate_model(model, tokenized_datasets, data_collator, device)
print(metrics)

{'accuracy': 0.8766857788904143, 'precision': 0.8762180840832915, 'recall': 0.8766857788904143, 'f1': 0.876300814105596}
